In [30]:
import pandas as pd 

albums = pd.read_csv("data/spotify_spotify_albums.csv", index_col=0) 
artists = pd.read_csv("data/spotify_spotify_artists.csv", index_col=0) 
tracks = pd.read_csv("data/spotify_spotify_tracks.csv",index_col=0)

In [42]:
albums.head() # We see first 5 rows across all columns. This looks to be a wide dataset with many columns containing categorical variables.
# Notice the first column: 'Unnamed: 0'. This is an index column that wasn't read correctly before we re-ran the previous block with 'index_col = 0'. 
# If we preferred not to re-read a large file like this, we could have typed:
  # albums.drop("Unnamed: 0", axis=1, inplace=True)
# Or:
  # albums = albums.iloc[:,1:] 

albums.tail() # Shows last 5 rows and lets us determine number of rows in dataframe (75511)
albums.shape # Gives dimensionality of dataframe

albums.loc[10:20,['name', 'release_date']]

,name,release_date
10,Beethoven: 6 Bagatelles & Piano Sonatas Nos. 3...,2019-03-01
11,Au Long de la Loire,2019-03-01
12,I Still Miss U,2019-03-15
13,Kolmekymppinen,1980
14,Bruja,2018-04-24
15,Sonatas for two violins,2019-03-01
16,Sirènes,2019-03-01
17,Light of Day,2019-03-01
18,Madame del Campo,2015-08-03
19,Beethoven: Piano Concerto No. 2 & Triple Conce...,2019-03-01


In [38]:
artists.head() # This has two quantitative variable columns in 'artist_popularity' and 'followers'.
# We get the artist name here but oddly enough not the artist id. We can't join this to albums on that basis, then.
artists.tail()
artists.shape

(56129, 8)

In [40]:
tracks.head() # Another wide dataset with multiple quantitative variables. joining with the other two sets could allow for interesting statistical analysis.
tracks.tail()
tracks.shape # Logically, there are more tracks than albums, and there are more albums than artists

(101939, 31)

,album_type,artist_id,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,track_id,track_name_prev,uri,type
0,single,3DiDSECUqqY1AuBP8qtaIa,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",{'spotify': 'https://open.spotify.com/album/1g...,https://api.spotify.com/v1/albums/1gAM7M4rBwEb...,1gAM7M4rBwEbSPeAQR2nx1,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",If I Ain't Got You EP,2019-02-08,day,6,2iejTMy9XZ8Gaae0aQ2yl0,track_32,spotify:album:1gAM7M4rBwEbSPeAQR2nx1,album
1,album,6s1pCNXcbdtQJlsnM1hRIA,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",{'spotify': 'https://open.spotify.com/album/4K...,https://api.spotify.com/v1/albums/4KfJZV7WfolY...,4KfJZV7WfolYlxBzOTo66s,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Shostakovich Symphony No.5 - Four Romances on ...,2019-03-01,day,8,1WQfghEjszJJ4H8MAWrQ2C,track_11,spotify:album:4KfJZV7WfolYlxBzOTo66s,album
2,single,5YjfNaHq05WrwldRe1QSBc,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",{'spotify': 'https://open.spotify.com/album/7n...,https://api.spotify.com/v1/albums/7nLYY7uAVUb5...,7nLYY7uAVUb57kpd7tZxnS,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Take My Bass,2019-03-14,day,1,3jJKj4QTK3v18ZSwpk7AcV,track_15,spotify:album:7nLYY7uAVUb57kpd7tZxnS,album
3,single,2G9Vc16JCpnZmK4uGH46Fa,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",{'spotify': 'https://open.spotify.com/album/6p...,https://api.spotify.com/v1/albums/6p20Rt4x2Qn5...,6p20Rt4x2Qn5mUMRi1s6pj,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Hypnotizing (Are U),2016-11-16,day,1,1xGtDafUZbHyYC3Xarcbrj,track_46,spotify:album:6p20Rt4x2Qn5mUMRi1s6pj,album
4,single,2dwM9OcE4c3Ph1UBINSodx,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",{'spotify': 'https://open.spotify.com/album/1X...,https://api.spotify.com/v1/albums/1XeoOqC1q7U2...,1XeoOqC1q7U2iyLEQJ64cu,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Sunshine,2018-07-20,day,1,0gWtsXvXOzAT6FtM3ur8in,track_10,spotify:album:1XeoOqC1q7U2iyLEQJ64cu,album
